In [ ]:
import tensorflow as tf
import numpy as np
import os
import cv2
import numpy as np
import random as rn
import tensorflow as tf
import matplotlib.pyplot as plt

global n_classes, b_count, b_count2, seq1, seq2
n_classes = 100
b_count = 0
b_count2 = 0
seq1 = np.asarray(np.zeros((50000)), np.int64)
for i in range(50000):
  seq1[i]=i
rn.shuffle(seq1)

seq2 = np.asarray(np.zeros((10000)), np.int64)
for i in range(10000):
  seq2[i]=i
rn.shuffle(seq2)

In [ ]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [ ]:
_weights = {
        'wc1': tf.Variable(tf.truncated_normal([7, 7, 3, 96], stddev=0.01)),
        'wc2': tf.Variable(tf.truncated_normal([5, 5, 96, 128], stddev=0.01)),
        'wc3': tf.Variable(tf.truncated_normal([3, 3, 128, 256], stddev=0.01)),
        'wc4': tf.Variable(tf.truncated_normal([3, 3, 256, 256], stddev=0.01)),
        'wc5': tf.Variable(tf.truncated_normal([3, 3, 256, 256], stddev=0.01)),
        'wd2': tf.Variable(tf.truncated_normal([512, 512], stddev=0.01)),
        'out': tf.Variable(tf.truncated_normal([512, n_classes], stddev=0.01))
    }
_biases = {
        'bc1': tf.Variable(tf.truncated_normal([96], stddev=0.1)),
        'bc2': tf.Variable(tf.truncated_normal([128], stddev=0.1)),
        'bc3': tf.Variable(tf.truncated_normal([256], stddev=0.1)),
        'bc4': tf.Variable(tf.truncated_normal([256], stddev=0.1)),
        'bc5': tf.Variable(tf.truncated_normal([256], stddev=0.1)),
        'bd2': tf.Variable(tf.truncated_normal([512], stddev=0.1)),
        'out': tf.Variable(tf.truncated_normal([n_classes], mean=0.1, stddev=0.1))
    }

In [ ]:
def relu(x,name="activation"):
    return tf.nn.relu(x, name=name)
    
def conv2d(name, l_input, w, b, s, p, scope):
    l_input = tf.nn.conv2d(l_input, w, strides=[1,s,s,1], padding=p, name=name)
    l_input = l_input+b
    l_input = relu(l_input)

    return l_input

def max_pool(name, l_input, k, s):
    return tf.nn.max_pool(l_input, ksize=[1, k, k, 1], strides=[1, s, s, 1], padding='VALID', name=name)

def norm(l_input, lsize=4, name="lrn"):
    return tf.nn.lrn(l_input, lsize, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name=name)


In [ ]:
   
def alex_net(_X, _dropout, batch_size):
    conv1 = conv2d('conv1', _X, _weights['wc1'], _biases['bc1'], 3, 'SAME', 'conv1')
    pool1 = max_pool('pool1', conv1, k=3,s=2)
    conv2 = conv2d('conv2', pool1, _weights['wc2'], _biases['bc2'], 1, 'SAME', 'conv2')
    pool2 = max_pool('pool2', conv2, k=3,s=2)
    conv3 = conv2d('conv3', pool2, _weights['wc3'], _biases['bc3'], 1, 'SAME', 'conv3')
    conv4 = conv2d('conv4', conv3, _weights['wc4'], _biases['bc4'], 1, 'SAME', 'conv4')
    conv5 = conv2d('conv5', conv4, _weights['wc5'], _biases['bc5'], 1, 'SAME', 'conv5')
    # Find current size of conv5 to fit the requirement of FC1.
    sizes = conv5.get_shape().as_list()
    neurons =  sizes[1]*sizes[2]*sizes[3]
    dense1 = tf.reshape(conv5, [batch_size, neurons]) # Reshape conv5 output to fit dense layer input
    wei_den1 = tf.Variable(tf.truncated_normal([neurons, 512], stddev=0.01))
    b_den1 = tf.Variable(tf.truncated_normal([512], stddev=0.1))
    
    dense1 = relu(tf.matmul(dense1, wei_den1) + b_den1, name='fc1') # Relu activation
    dd1=tf.nn.dropout(dense1, _dropout)
    dense2 = relu(tf.matmul(dd1, _weights['wd2']) + _biases['bd2'], name='fc2') # Relu activation
    dd2=tf.nn.dropout(dense2, _dropout)
    out = tf.matmul(dd2, _weights['out']) + _biases['out'] # Relu activation

    print("C1:",conv1.get_shape())
    print("P1:",pool1.get_shape())
    print("C2:",conv2.get_shape())
    print("P2:",pool2.get_shape())
    print("C3:",conv3.get_shape())
    print("C4:",conv4.get_shape())
    print("C5:",conv5.get_shape())
    print("d1:",dense1.get_shape())
    print("d2:",dense2.get_shape())  
    
    
    return out

In [ ]:
def Rotate(img,angle):
    M = cv2.getRotationMatrix2D((16,16),angle,1.)
    img = cv2.warpAffine(img,M,(32,32))
    return img

def trans(img,mode):
    if(mode == 0):
        M = np.float32([[1, 0, 0], [0, 1, 8]])
        img = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))
    elif(mode == 1):
        M = np.float32([[1, 0, 0], [0, 1, -8]])
        img = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))
    elif(mode == 2):
        M = np.float32([[1, 0, 8], [0, 1, 0]])
        img = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))
    elif(mode == 3):
        M = np.float32([[1, 0, -8], [0, 1, 0]])
        img = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))
    return img

def data_augmentation(img,epoch1):
    if epoch1 > 1:
        rnd = rn.randint(0,27)
        for i in range(len(img)):
            im1 = img[i]
            if (rnd == 0):
                im1 = Rotate(im1,30)
            elif (rnd == 1):
                im1 = Rotate(im1,60)
            elif (rnd == 2):
                im1 = Rotate(im1,90)
            elif (rnd == 3):
                im1 = Rotate(im1,120)
            elif (rnd == 4):
                im1 = Rotate(im1,-30)
            elif (rnd == 5):
                im1 = Rotate(im1,-60)
            elif (rnd == 6):
                im1 = Rotate(im1,-90)                
            elif (rnd == 7):
                im1 = Rotate(im1,-120)   
            elif (rnd == 8):
                im1 = cv2.flip(im1,1)
            elif (rnd == 9):
                im1 = cv2.flip(im1,0)
            elif (rnd == 10):
                im1 = cv2.flip(im1,-1)
            elif (rnd == 11):
                im1 = trans(im1,0)
            elif (rnd == 12):
                im1 = trans(im1,1)
            elif (rnd == 13):
                im1 = trans(im1,2)
            elif (rnd == 13):
                im1 = trans(im1,3)
            img[i] = im1
    return img

In [ ]:
def getvalbatch(data, labels, glen1):
    global b_count2, seq2
    b_count = b_count2
    idx = (b_count*batch_size)%glen1
    bx=[]
    by=[]
    for i in range(batch_size):
        idx2 = (idx+i)%glen1
        idx2 = seq2[idx2]%glen1
        im1 = np.reshape(data[idx2, :], [3,32,32]).transpose(1,2,0)
        
        im1 = cv2.normalize(im1,None,0,1,cv2.NORM_MINMAX,cv2.CV_32F)
        bx.append(im1)
        by.append(labels[idx2])
    bx = np.reshape(np.asarray(bx), [batch_size, 32, 32, 3])
    by = np.asarray(by)
    b_count2 +=1
    return bx, by

def getbatch(data, labels, glen1,epoch1):
    global b_count, seq1
    idx = (b_count*batch_size)%glen1
    bx=[]
    by=[]
    for i in range(batch_size):
        idx2 = (idx+i)%glen1
        idx2 = seq1[idx2]%glen1
        im1 = np.reshape(data[idx2, :], [3,32,32]).transpose(1,2,0)
           
        im1 = cv2.normalize(im1,None,0,1,cv2.NORM_MINMAX,cv2.CV_32F)
        bx.append(im1)
        by.append(labels[idx2])
    bx = np.reshape(np.asarray(bx), [batch_size, 32, 32, 3])
    bx = data_augmentation(bx,epoch1)
    by = np.asarray(by)
    b_count +=1
    return bx, by

def unpickle(file):
    # Python 3 version
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    #---------------------------------------------
    # Python 2 version
    #import cPickle
    #with open(file, 'rb') as fo:
    #    dict = cPickle.load(fo)
    #---------------------------------------------
    return dict

In [ ]:
# Training setting
batch_size = 300 
display_step = 400
n_classes = 100 # # of classes
dropout = 0.8 # Dropout rate

In [ ]:
keep_prob = tf.placeholder(tf.float32)          # Dropout rate to be fed
learning_rate = tf.placeholder(tf.float32)      # Learning rate to be fed
tX = tf.placeholder("float", [batch_size, 32, 32, 3]) # Training data batch
tY = tf.placeholder("int64", [batch_size])            # Training label batch
lr = 1e-3                                # Learning rate start
print(lr)

dict1=unpickle("data/train")
data = np.asarray(dict1[b"data"], np.float32)
labels = dict1[b"fine_labels"]
glen1 = len(labels)
training_iters = glen1 * 1000

dict1=unpickle("data/test")
data2 = np.asarray(dict1[b"data"], np.float32)
labels2 = dict1[b"fine_labels"]
tlen1 = len(data2)

print("Total labels=%d"%(max(labels)+1))

print("Found %d training images & %d test images"%(glen1, tlen1))

pred = alex_net(tX,keep_prob,batch_size)

cost = tf.losses.sparse_softmax_cross_entropy(labels=tY, logits=pred)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
correct_prediction=tf.reduce_mean(tf.cast(tf.nn.in_top_k(pred, tY, 5), tf.float32)) #top-k
#correct_prediction = tf.equal(tf.argmax(pred, 1), tY)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
saver = tf.train.Saver()
init = tf.global_variables_initializer()
train_accuracy = []
test_accuracy = []
with tf.Session() as sess:
    sess.run(init)
    step = 0
    while step * batch_size < training_iters:
        epoch1=np.floor((step*batch_size)/glen1)
        bx, by = getbatch(data, labels, glen1,epoch1)
        
        if (epoch1 > 500) & (lr == 1e-3):
            lr = lr / 10
            
        # Get a batch
        sess.run([optimizer],  feed_dict={tX:bx, tY:by, keep_prob: dropout, learning_rate: lr})
        
        if (step % 15000==1) & (step>15000):
            save_path = saver.save(sess, "tf_alex_model_iter" + str(step) + ".ckpt")
            print("Model saved in file at iteration %d: %s" % (step*batch_size,save_path))

        if step % display_step == 1:
            # calculate the loss

            loss = sess.run(cost, feed_dict={tX:bx, tY:by, keep_prob: 1.})
            acc = sess.run(accuracy, feed_dict={tX:bx, tY:by,keep_prob: 1.})
            train_accuracy.append(acc)
            acc2 = []
            for j in range(int(tlen1 / batch_size)):
                tbx, tby = getvalbatch(data2, labels2, tlen1)
                acc2.append(sess.run(accuracy, feed_dict={tX:tbx, tY:tby,keep_prob: 1.}))
            acc2 = np.mean(acc2)
            
            print("Iter=%d/epoch=%d, Loss=%.6f, Training Accuracy=%.6f, Test Accuracy=%.6f, lr=%f" 
                  % (step*batch_size, epoch1 ,loss, acc, acc2, lr))
            test_accuracy.append(acc2)

        step += 1
    print("Optimization Finished!")
    save_path = saver.save(sess, "tf_alex_model.ckpt")
print("Model saved in file: %s" % save_path)